In [ ]:
import json
import requests
import pandas as pd
from owlready2 import *
from tqdm import tqdm
import os
import urllib.request 

local_path = '/Users/gburns/Documents/2022H2/pathogen_catalog'

class OwlSparqlUtils: 
  '''
  Interactions with OWL-based Ontologies 
  '''
  def __init__(self, local_path, file_name, remote_url):
    self.local_path = local_path
    self.owl_path = local_path+'/'+file_name
    if os.path.exists(self.owl_path) is False:
      print('Downloading latest version of '+file_name)
      urllib.request.urlretrieve(doid_url, self.owl_path)
    self.ontology = get_ontology(self.owl_path).load()
      
owlsparql = OwlSparqlUtils(local_path, 'doid.owl', doid_url)
owlsparql

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


NameError: name 'doid_url' is not defined

In [ ]:
from owlready2 import *
doid_url = 'http://purl.obolibrary.org/obo/doid/doid-merged.owl'
o = get_ontology(doid_url)
o.load()

get_ontology("http://purl.obolibrary.org/obo/doid/doid-merged.owl#")

In [ ]:
sparql = '''PREFIX doid: <http://purl.obolibrary.org/obo/doid.owl#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>
SELECT DISTINCT ?doid ?a ?b
WHERE {
     ?doid rdfs:subClassOf ?a .
     ?a ?b obo:IDO_0000664 .
}'''

doid = get_namespace("http://purl.obolibrary.org/obo/doid.owl#")
obo = get_namespace("http://purl.obolibrary.org/obo/")
m = re.search('SELECT DISTINCT (.*)\n', sparql)
if m is not None:
  col_headings = m.group(1).split(' ')
else:
  raise Exception("Can't read column headings in " + sparql )  

l = [i for i in default_world.sparql(sparql)]
df = pd.DataFrame(l, columns=col_headings)
df

,?doid,?a,?b
0,obo.DOID_9986,obo.IDO_0000664.some(obo.CL_0000542),18
1,obo.DOID_9965,obo.IDO_0000664.some(obo.NCBITaxon_5811),18
2,obo.DOID_992,obo.IDO_0000664.some(obo.NCBITaxon_12542),18
3,obo.DOID_9790,obo.IDO_0000664.some(obo.NCBITaxon_6264),18
4,obo.DOID_9784,obo.IDO_0000664.some(obo.NCBITaxon_6333),18
...,...,...,...
9274,obo.DOID_0050012,obo.IDO_0000664.some(obo.NCBITaxon_37124),18
9275,obo.DOID_0040093,obo.IDO_0000664.some(obo.CHEBI_24431 & obo.RO_...,18
9276,obo.DOID_0040086,obo.IDO_0000664.some(obo.NCBITaxon_1891762),18
9277,obo.DOID_0040084,obo.IDO_0000664.some(obo.NCBITaxon_1313),18


In [ ]:
sparql = '''PREFIX doid: <http://purl.obolibrary.org/obo/doid.owl#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>
SELECT DISTINCT ?doid ?restriction ?b
WHERE {
     ?doid rdfs:subClassOf ?restriction .
     ?restriction ?b obo:IDO_0000664 .
}'''

doid = get_namespace("http://purl.obolibrary.org/obo/doid.owl#")
obo = get_namespace("http://purl.obolibrary.org/obo/")
m = re.search('SELECT DISTINCT (.*)\n', sparql)
if m is not None:
  col_headings = m.group(1).split(' ')
else:
  raise Exception("Can't read column headings in " + sparql )  

l = [i for i in default_world.sparql(sparql)]
df = pd.DataFrame(l, columns=col_headings)
df

,?doid,?restriction,?b
0,obo.DOID_9986,obo.IDO_0000664.some(obo.CL_0000542),18
1,obo.DOID_9965,obo.IDO_0000664.some(obo.NCBITaxon_5811),18
2,obo.DOID_992,obo.IDO_0000664.some(obo.NCBITaxon_12542),18
3,obo.DOID_9790,obo.IDO_0000664.some(obo.NCBITaxon_6264),18
4,obo.DOID_9784,obo.IDO_0000664.some(obo.NCBITaxon_6333),18
...,...,...,...
9274,obo.DOID_0050012,obo.IDO_0000664.some(obo.NCBITaxon_37124),18
9275,obo.DOID_0040093,obo.IDO_0000664.some(obo.CHEBI_24431 & obo.RO_...,18
9276,obo.DOID_0040086,obo.IDO_0000664.some(obo.NCBITaxon_1891762),18
9277,obo.DOID_0040084,obo.IDO_0000664.some(obo.NCBITaxon_1313),18


In [ ]:
from owlready2 import *
doid_url = 'http://purl.obolibrary.org/obo/doid/doid-merged.owl'
o = get_ontology(doid_url)
o.load()

pathogen_list = []
for c in list(o.classes()):
    if len(c.IDO_0000664)>0:
        for m in c.IDO_0000664:
            taxa = []
            if isinstance(m, owlready2.class_construct.Or):
                taxa = [t for t in m._Classes if str(t)[0:14]=='obo.NCBITaxon_']
            elif isinstance(m, owlready2.class_construct.And):
                taxa = [t for t in m._Classes if str(t)[0:14]=='obo.NCBITaxon_']
            elif str(m)[0:14]=='obo.NCBITaxon_': 
                taxa = [m]
            for t in taxa: 
                pathogen_list.append((c, c.label[0], t, t.label[0]))
df = pd.DataFrame(pathogen_list, columns=['doid','disease_label','ncbiTaxon_id', 'species_label'])
df

,doid,disease_label,ncbiTaxon_id,species_label
0,obo.DOID_0040083,Chlamydia pneumonia,obo.NCBITaxon_83558,Chlamydia pneumoniae
1,obo.DOID_874,bacterial pneumonia,obo.NCBITaxon_2,Bacteria
2,obo.DOID_0040084,Streptococcus pneumonia,obo.NCBITaxon_1313,Streptococcus pneumoniae
3,obo.DOID_104,bacterial infectious disease,obo.NCBITaxon_2,Bacteria
4,obo.DOID_0040086,Polyomavirus-associated nephropathy,obo.NCBITaxon_1891762,Betapolyomavirus hominis
...,...,...,...,...
321,obo.DOID_9682,yellow fever,obo.NCBITaxon_11089,Yellow fever virus
322,obo.DOID_9700,bacterial conjunctivitis,obo.NCBITaxon_2,Bacteria
323,obo.DOID_9784,trichinosis,obo.NCBITaxon_6333,Trichinella
324,obo.DOID_9790,toxocariasis,obo.NCBITaxon_6264,Toxocara


In [ ]:
df.to_csv(local_path+'/doid_pathogens.txt', sep='\t')

In [ ]:
o.search(subclass_of

In [ ]:
print(obo.DOID_4.label)

['disease', 'disease', 'disease']


In [ ]:
MONDO_LATEST_URL = 'https://github.com/monarch-initiative/mondo/releases/download/v2022-09-06/mondo.owl'
